In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
import os

In [6]:
train='/Datasets/Soldier/train'
#os.mkdir(train)

In [7]:
valid = '/Datasets/Soldier/Valid'
#os.mkdir(valid)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from shutil import copyfile

# TF Model 

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_gen = train_datagen.flow_from_directory(
    train,
    batch_size=64,
    class_mode='binary',
    target_size=(300, 300)
)

valid_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'

)
valid_gen = valid_datagen.flow_from_directory(
    valid,
    batch_size=64,
    class_mode='binary',
    target_size=(300, 300)
)

Found 63 images belonging to 2 classes.
Found 29 images belonging to 2 classes.


In [13]:
# model creation
activation='relu'

model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(64,(5,5),activation=activation,input_shape=(300,300,3)))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#2nd CNN
model1.add(tf.keras.layers.Conv2D(32,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#3rd CNN
model1.add(tf.keras.layers.Conv2D(16,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#4th CNN
model1.add(tf.keras.layers.Conv2D(8,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(1,activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 296, 296, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 144, 32)      51232     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 68, 68, 16)        12816     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 34, 34, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 8)         3

In [17]:
optimizer='adam'
loss = 'binary_crossentropy'
metrics=['accuracy']
model1.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [18]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_acc') >0.8):
            print('Achieved 80% accuracy')
            self.model.stop_training=True
callback = mycallback()    

In [19]:
history = model1.fit(train_gen,epochs=5,verbose=2,validation_data=valid_gen,steps_per_epoch=10,callbacks=[callback])

Epoch 1/5
10/10 - 289s - loss: 0.6179 - acc: 0.6984 - val_loss: 0.7238 - val_acc: 0.5517
Epoch 2/5
10/10 - 288s - loss: 0.4718 - acc: 0.7905 - val_loss: 1.2448 - val_acc: 0.5517
Epoch 3/5
10/10 - 301s - loss: 0.3297 - acc: 0.8635 - val_loss: 1.6601 - val_acc: 0.5517
Epoch 4/5
10/10 - 292s - loss: 0.2538 - acc: 0.8794 - val_loss: 2.2683 - val_acc: 0.5517
Epoch 5/5
10/10 - 257s - loss: 0.2487 - acc: 0.8873 - val_loss: 1.4556 - val_acc: 0.6207
